In [1]:
import os
import csv
import shutil
import pandas as pd

### Consolidating the initial data

In [2]:
# Define the function merge_csv_files that takes an input_folder and an output_file as arguments
def merge_csv_files(input_folder, output_file):
    # Use list comprehension to get a list of all .csv files in the input_folder
    file_list = [file for file in os.listdir(input_folder) if file.endswith('.csv')]
    
    # Initialize an empty DataFrame to store the merged data
    merged_data = pd.DataFrame()
    
    # Loop through each file in the file_list
    for file in file_list:
        # Construct the full path of the file by joining the input_folder and file name
        file_path = os.path.join(input_folder, file)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, index_col=0)
        
        # Concatenate the newly read DataFrame with the existing merged_data DataFrame along axis=1 (columns)
        merged_data = pd.concat([merged_data, df], axis=1)

    # Reorder the columns of the merged_data DataFrame.
    # If column names are integers, they are sorted numerically.
    # Otherwise, they remain in their original order.
    merged_data = merged_data.reindex(
        sorted(merged_data.columns, key=lambda x: int(x) if str(x).isdigit() else x), 
        axis=1
    )

    # Write the merged DataFrame to a new CSV file specified by output_file
    merged_data.to_csv(output_file)

In [14]:
# Call the merge_csv_files function to merge all CSV files in the 'Base2_Nrep50' folder
# and save the resulting merged DataFrame to a file named 'Nrep50.csv'
merge_csv_files('Base2_Nrep50', 'Nrep50.csv')

### The data is converted to a txt format

In [4]:
# Define the function save_as_txt that takes a csv_file and a txt_file as arguments
def save_as_txt(csv_file, txt_file):
    # Open the csv_file in read mode ('r')
    with open(csv_file, 'r') as file:
        # Create a csv reader object to read the CSV data
        csv_data = csv.reader(file)
        
        # Read the header row from the CSV file
        header = next(csv_data) 
        
        # Read the remaining data into a list
        data = list(csv_data)

    # Open the txt_file in write mode ('w')
    with open(txt_file, 'w') as file:
        # Loop through each column in the header except the first one
        
        for i in range(len(header) - 1):
            # Write the header name preceded by a colon and followed by a newline
            file.write(f':{header[i+1]}\n') 
            
            # Write the data for that column, separated by commas, followed by a newline
            file.write(','.join([row[i+1] for row in data]) + '\n')

In [ ]:
# Call the save_as_txt function to convert the 'Nrep50.csv' file into a text file named 'Nrep50.txt'
save_as_txt('Nrep50.csv', 'Nrep50.txt')

### Generating data.txt

In [26]:
# Define a function called merge_txt_files that takes a list of text files to be merged (file_list)
# and the name of the output file (output_file) where the merged content will be saved
def merge_txt_files(file_list, output_file):
    # Open the output file in write mode ('w')
    with open(output_file, 'w') as outfile:
        
        # Loop through each file in the list of text files to be merged
        for file_name in file_list:
            # Open the current text file in read mode ('r')
            with open(file_name, 'r') as infile:
                # Read the entire content of the current text file into a string
                content = infile.read()
                
                # Write the content of the current text file to the output file
                outfile.write(content)

In [27]:
# Create a list of text file names that are to be merged
file_list = ['Nrep500.txt', 'Nrep450.txt', 'Nrep400.txt', 'Nrep350.txt', 'Nrep300.txt', 'Nrep250.txt', 'Nrep200.txt', 'Nrep150.txt', 'Nrep100.txt', 'Nrep50.txt']

# Specify the name of the output file where the merged content will be saved
output_file = 'data.txt'

# Call the merge_txt_files function to merge all text files in the file_list
# and save the resulting merged content into a file named 'data.txt'
merge_txt_files(file_list, output_file)

### Computing the value of omega0

In [28]:
# Define a function called calculate_averages that takes a single argument, file_path,
# which is the path to the text file containing the data
def calculate_averages(file_path):
    # Open the text file in read mode ('r') with UTF-8 encoding
    with open(file_path, 'r', encoding='UTF-8') as file:
        # Read all lines from the file into a list
        lines = file.readlines()

    # Initialize an empty list to store the data    
    data = []
    
    # Initialize variables to store the sum and count for calculating omega0 average
    omega0_sum = 0
    count = 0
    
    # Loop through each line in the file
    for line in lines:
        # Remove leading and trailing whitespaces
        line = line.strip()
        
        # If the line starts with a colon, it indicates the index
        if line.startswith(':'):
            index = int(line[1:])
        # If the line has content (i.e., is not empty)
        elif line:
            try:
                # Convert the comma-separated string of numbers into a list of integers
                values = [int(x) for x in line.split(',')]
                
                # Calculate the average of the values
                avg = sum(values) / len(values)
                
                # Append the index and average value as a tuple to the data list
                data.append((index, avg))
                
                # Update the sum and count for calculating omega0 average
                omega0_sum += avg
                count += 1
                
            # Handle ValueError (e.g., if the line contains non-integer values)
            except ValueError:
                continue

    # Calculate the average of omega0
    omega0_avg = omega0_sum / count

    # Create an output CSV file name based on the input text file name
    output_file = file_path.split('.')[0] + '.csv'
    
    # Open the output CSV file in write mode ('w')
    with open(output_file, 'w', newline='') as csv_file:
        # Create a CSV writer object
        writer = csv.writer(csv_file)
        
        # Write the header row to the CSV file
        writer.writerow(['size', 'omega0'])
        
        # Loop through the data list and write each row to the CSV file
        for index, avg in data:
            writer.writerow([index, avg])

In [29]:
# Call the calculate_averages function to process the data in 'data.txt' 
# and write the calculated averages to a new CSV file
calculate_averages('data.txt')